In [1]:
import requests
import pandas as pd

# Lấy messages từ Kafka Consumer Service
response = requests.get('http://localhost:8004/messages/recent?limit=100')
data = response.json()

# Convert to DataFrame
df = pd.DataFrame(data['messages'])

print(f"✅ Loaded {len(df)} messages from Kafka buffer")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nDataFrame shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

# Xem thống kê
print(f"\n📊 Statistics:")
print(f"  - Unique users: {df['user_name'].nunique()}")
print(f"  - Unique zones: {df['zone_name'].nunique()}")
print(f"  - Status distribution:\n{df['status'].value_counts()}")


✅ Loaded 100 messages from Kafka buffer

Columns: ['user_id', 'user_name', 'camera_id', 'zone_id', 'zone_name', 'iop', 'threshold', 'status', 'timestamp', 'frame_id', 'violation_duration', 'missing_count', 'required_count', 'violation_start_time']

DataFrame shape: (100, 14)

First few rows:
  user_id user_name  camera_id zone_id           zone_name  iop  threshold  \
0       2   KHIEM_2          0   zone1     Zone_KHIEM_CAM1  0.5       60.0   
1       3      Dang          0   zone3  Zone_DANG_CAM_1905  0.5       60.0   
2       2   KHIEM_2          0   zone1     Zone_KHIEM_CAM1  0.5       60.0   
3       3      Dang          0   zone3  Zone_DANG_CAM_1905  0.5       60.0   
4       2   KHIEM_2          0   zone1     Zone_KHIEM_CAM1  0.5       60.0   

                 status                   timestamp  frame_id  \
0  violation_incomplete  2025-11-20T08:43:22.997931      2450   
1  violation_incomplete  2025-11-20T08:43:22.999650      2450   
2  violation_incomplete  2025-11-20T08:43:2

In [2]:
import pandas as pd
from utils.kafka_manager import KafkaAlertConsumer
import threading
import time

# List để lưu messages
messages_list = []

def collect_message(msg):
    """Callback để collect messages"""
    messages_list.append(msg)
    print(f"📨 Collected {len(messages_list)} messages", end='\r')

# Khởi tạo consumer
consumer = KafkaAlertConsumer(
    bootstrap_servers='localhost:9092',
    topic='person_reid_alerts',
    group_id='dataframe_consumer_group',  # Group ID mới
    enable=True
)

# Nếu muốn đọc từ đầu topic, modify config:
from confluent_kafka import Consumer
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'dataframe_consumer_group',  # Group ID mới
    'auto.offset.reset': 'earliest',  # Đọc từ đầu
    'enable.auto.commit': True,
}
consumer.consumer = Consumer(conf)
consumer.consumer.subscribe(['person_reid_alerts'])

print("🔄 Starting consumer...")
print("   (Will collect messages for 30 seconds)")

# Start consumer thread
stop_flag = False

def consume_loop():
    while not stop_flag:
        msg = consumer.consumer.poll(timeout=1.0)
        if msg and not msg.error():
            import json
            try:
                message_dict = json.loads(msg.value().decode('utf-8'))
                collect_message(message_dict)
            except:
                pass

thread = threading.Thread(target=consume_loop, daemon=True)
thread.start()

# Collect trong 30 giây
time.sleep(30)
stop_flag = True
thread.join(timeout=5)

print(f"\n✅ Collected {len(messages_list)} messages")

# Convert to DataFrame
df = pd.DataFrame(messages_list)

print(f"\n📊 DataFrame Info:")
print(df.info())
print(f"\nFirst 5 rows:")
print(df.head())

2025-11-20 15:48:28.584 | INFO     | utils.kafka_manager:_ensure_topic_exists:245 - ℹ️ Topic 'person_reid_alerts' already exists
2025-11-20 15:48:28.588 | INFO     | utils.kafka_manager:__init__:222 - ✅ Kafka Consumer initialized: localhost:9092 <- topic 'person_reid_alerts' (group: dataframe_consumer_group)


🔄 Starting consumer...
   (Will collect messages for 30 seconds)
📨 Collected 6577 messages
✅ Collected 6577 messages

📊 DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6577 entries, 0 to 6576
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   user_id               6577 non-null   object 
 1   user_name             6577 non-null   object 
 2   camera_id             6577 non-null   int64  
 3   zone_id               6577 non-null   object 
 4   zone_name             6577 non-null   object 
 5   iop                   6577 non-null   float64
 6   threshold             6577 non-null   float64
 7   status                6577 non-null   object 
 8   timestamp             6577 non-null   object 
 9   frame_id              6577 non-null   int64  
 10  violation_duration    6577 non-null   float64
 11  missing_count         6577 non-null   int64  
 12  required_count        6577 non-null   

In [3]:
import polars as pl 
from hvplot import polars as hvpolars
import hvplot.polars

In [4]:
df_pl = pl.from_pandas(df)
df_pl

user_id,user_name,camera_id,zone_id,zone_name,iop,threshold,status,timestamp,frame_id,violation_duration,missing_count,required_count,violation_start_time
str,str,i64,str,str,f64,f64,str,str,i64,f64,i64,i64,f64
"""2""","""KHIEM_2""",0,"""zone1""","""Zone_KHIEM_CAM1""",0.5,60.0,"""violation_incomplete""","""2025-11-20T08:37:31.265408""",0,0.0,1,1,0.0
"""4""","""Thanh""",0,"""zone2""","""ZONE_THANH_CAM_1905""",0.5,60.0,"""violation_incomplete""","""2025-11-20T08:37:31.265929""",0,0.0,1,1,0.0
"""3""","""Dang""",0,"""zone3""","""Zone_DANG_CAM_1905""",0.5,60.0,"""violation_incomplete""","""2025-11-20T08:37:31.266199""",0,0.0,1,1,0.0
"""2""","""KHIEM_2""",0,"""zone1""","""Zone_KHIEM_CAM1""",0.5,60.0,"""violation_incomplete""","""2025-11-20T08:37:31.373960""",1,0.04,1,1,0.0
"""4""","""Thanh""",0,"""zone2""","""ZONE_THANH_CAM_1905""",0.5,60.0,"""violation_incomplete""","""2025-11-20T08:37:31.374590""",1,0.04,1,1,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""3""","""Dang""",0,"""zone3""","""Zone_DANG_CAM_1905""",0.5,60.0,"""violation_incomplete""","""2025-11-20T08:43:30.383115""",2497,99.88,1,1,0.0
"""2""","""KHIEM_2""",0,"""zone1""","""Zone_KHIEM_CAM1""",0.5,60.0,"""violation_incomplete""","""2025-11-20T08:43:30.480202""",2498,99.92,1,1,0.0
"""3""","""Dang""",0,"""zone3""","""Zone_DANG_CAM_1905""",0.5,60.0,"""violation_incomplete""","""2025-11-20T08:43:30.481042""",2498,99.92,1,1,0.0


In [9]:
df_khiem_cam1 = df_pl.filter(
    pl.col("user_id")=="2",
    pl.col("user_name")!="Person 2",
    pl.col("camera_id")==0,
    pl.col("zone_id")=="zone5"
).with_columns(
    pl.col("timestamp").cast(pl.Datetime).dt.truncate("20s").alias("timestamp")
).group_by(
    pl.col("timestamp")
).agg(
    pl.sum("violation_duration").alias("violation_duration")
)
df_khiem_cam1

timestamp,violation_duration
datetime[μs],f64


%4|1763628839.354|MAXPOLL|rdkafka#consumer-3| [thrd:main]: Application maximum poll interval (300000ms) exceeded by 492ms (adjust max.poll.interval.ms for long-running message processing): leaving group


In [ ]:
df_khiem_cam1.hvplot(
    x="timestamp",
    y="violation_duration",
    kind="bar",
    title="violation_duration over time for Khiem in Camera 2",
    width=800,
    height=500
)

:Bars   [timestamp]   (violation_duration)